Note-The code was combinely used on colab and VScode as some of the snippets were working on different IDEs. 

# **1) Downloading the dataset**

I have downloaded whole data in zip format in My Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip gdrive/My\ Drive/coco_minitrain_25k.zip


We downloaded the dataset using given link-https://github.com/giddyyupp/coco-minitrain loaded into temporary cloud storage.



# **2) Peforming object detection via pretrained model**

You need to download following requirements via the given link- https://drive.google.com/file/d/19nwpRA1c_eJVa9U9eBJjq7OUa-54cLSU/view?usp=sharing which is executed in the given code.



---



In [ ]:
# Installing all the requirements at first
!pip install -r /content/gdrive/MyDrive/requirement.txt

In [5]:
import pixellib


In [ ]:
# Note: If this code doesn't executes, make sure that runtime is on GPU

from pixellib.instance import instance_segmentation


## Finding all the images in the given JSON file

In [9]:
import pandas as pd
annotation_file=pd.read_json(r'/content/coco_minitrain_25k/annotations/instances_val2017.json',lines=True)

In [10]:
Images=annotation_file['images']

In [11]:
single_info=Images[0]

In [ ]:
single_info

In [13]:
filenames=[]
for i in single_info:
  filenames.append(i['file_name'])

In [ ]:
filenames # This list wil be used in performing Object detection for each images of our dataset.

In [ ]:
segment_image = instance_segmentation()


The pretrained model is required to peform the object detection. It can downloaded via the given link-https://github.com/matterport/Mask_RCNN/releases/tag/v2.0

In [ ]:
# Loading the pre-trained model-- Mask RCNN which we will use.
segment_image.load_model("/content/gdrive/MyDrive/mask_rcnn_coco.h5") 


## Trying out our pretrained model to perform the detection task on one single image of our dataset to see the output.

### It will give outputed image consisting of bounding box in our current working directory by name-'output4.jpg'

In [ ]:
# Here a we see 'show_bboxes=True' that will highlight bounding boxes for our case
segment_image.segmentImage("/content/gdrive/MyDrive/000000000692.jpg", show_bboxes = True, output_image_name = "output4.jpg")

### Performing the same step but for each image of our dataset in order to get the bounding boxes in alll of them.

In [ ]:
from fileinput import filename
import numpy as np
from PIL import Image
from datetime import datetime
import flask
import json
import codecs
from flask import Flask, request, render_template
from pathlib import Path
import base64
import glob

for img in Path(r'/content/coco_minitrain_25k/images/train2017').glob("*.json"):
  segment_image.segmentImage(img, show_bboxes = True, output_image_name = "output.jpg")

  #Path(r"D:/ImageSearchengine/static/feature")

As executed above code, we are able to get the bounding boxes for each image in the dataset and those object detected images are automatically downloaded to our temporary google cloud storage.

# **3) Reverse Image Search - Feature Extraction**

In [ ]:
!pip install feature_extractor
from feature_extractor import FeatureExtractor

In [ ]:
fe = FeatureExtractor()
features = []
img_paths = []
for feature_path in Path(r"/content/coco_minitrain_25k/images/train2017").glob("*.json"):
    obj_text=codecs.open(feature_path,'r').read()
    features.append(json.loads(obj_text))
features = np.array(features)


That's how we get the features out

There is another method to get features out using a pre built user defined function which is as follows:

In [31]:
def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in ['num_detections', 'detection_boxes', 'detection_scores', 'detection_classes',
                        'detection_masks', 'detection_features']:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name( tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks( detection_masks, detection_boxes, image.shape[1], image.shape[2])
                detection_masks_reframed = tf.cast( tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims( detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict, feed_dict={image_tensor: image})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[ 'detection_classes'][0].astype(np.int64)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            output_dict['detection_features'] = output_dict['detection_features'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

# **4) Reverse Image Search - Similarity Search**


### Here we are deciding to use Cosine Similarity Search Metric to execute similarity search.

In [18]:
from sklearn.metrics.pairwise import cosine_similarity 


In [ ]:
# compute cosine similarities between images

cosSimilarities = cosine_similarity(features)

In [20]:
nb_closest_images = 5 
imgs_model_width = 224
imgs_model_height = 224
import matplotlib.pyplot as plt


In [21]:
def retrieve_most_similar_products(given_img):

  print("-----------------------------------------------------------------------")
  print("original product:")

  original = image.load_img(given_img, target_size=(imgs_model_width, imgs_model_height))
  plt.imshow(original)
  plt.show()
  print("-----------------------------------------------------------------------")
  print("most similar products:")

  closest_imgs = cos_similarities_df[given_img].sort_values(ascending=False)[1:nb_closest_images+1].index
  closest_imgs_scores = cos_similarities_df[given_img].sort_values(ascending=False)[1:nb_closest_images+1]

  for i in range(0,len(closest_imgs)):
      original = image.load_img(closest_imgs[i], target_size=(imgs_model_width, imgs_model_height))
      plt.imshow(original)
      plt.show()
      print("similarity score : ",closest_imgs_scores[i])

By the above User defined function, you can query the by one image to get similar results.